La procedure fait les choses suivantes:
- telecharge tous les datasets de datagouv
- avec des filtres "intelligents" selectionne les datasets qui parlent de subventions aux associations
- telecharge tous les fichiers de datagouv et selectionne seulement les fichiers des datasets isoles dans l'etape precedente
- telecharge tous les fichiers et les met dans une meme dataframe avec le schema attendu

In [ ]:
import requests
import pandas as pd
from IPython.display import FileLink
from io import StringIO
import csv

In [162]:
#telecharge tous les datasets de datagouv

url = 'https://www.data.gouv.fr/fr/datasets/r/f868cca6-8da1-4369-a78d-47463f19a9a3'

response = requests.get(url)

# vérifier le type de contenu de la réponse
content_type = response.headers.get('content-type')

if 'json' in content_type:
    # La réponse est au format JSON, on peut utiliser response.json()
    data = response.json()
    # code pour travailler avec les données JSON ici

elif 'csv' in content_type:
    # La réponse est au format CSV
    # Utiliser la fonction pandas.read_csv() pour lire les données CSV dans un dataframe
    data = pd.read_csv(url,delimiter=';')
    # code pour travailler avec le dataframe ici
    #print(data.head())
else:
    # Le type de contenu n'est pas géré
    print("Type de contenu non géré : ", content_type)


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (3,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [149]:
data.head()

,id,title,slug,acronym,url,organization,organization_id,description,frequency,license,...,resources_count,harvest.backend,harvest.domain,harvest.created_at,harvest.modified_at,quality_score,metric.discussions,metric.reuses,metric.followers,metric.views
0,6448a8817c8f7ead28c148e0,Lieux de médiation numérique sur le territoire...,lieux-de-mediation-numerique-sur-le-territoire...,NaN,http://www.data.gouv.fr/fr/datasets/lieux-de-m...,Cartographie Nationale des lieux de médiation ...,636cd3d7336141e7d0577f56,Lieux de médiation numérique proposant un acco...,daily,Licence Ouverte / Open Licence version 2.0,...,4,NaN,NaN,NaN,NaN,1.00,0,0,0,0
1,6448a87e3cbb852f0c4ff881,Lieux de médiation numérique sur le territoire...,lieux-de-mediation-numerique-sur-le-territoire...,NaN,http://www.data.gouv.fr/fr/datasets/lieux-de-m...,Cartographie Nationale des lieux de médiation ...,636cd3d7336141e7d0577f56,Lieux de médiation numérique proposant un acco...,daily,Licence Ouverte / Open Licence version 2.0,...,4,NaN,NaN,NaN,NaN,1.00,0,0,0,0
2,644893ec16d85892112ca03c,Parkings vélos gérés par Rennes Métropole,parkings-velos-geres-par-rennes-metropole-1,NaN,http://www.data.gouv.fr/fr/datasets/parkings-v...,Rennes Métropole en accès libre,580735be88ee3814295ff490,Cette couche de points permet de connaître la ...,unknown,License Not Specified,...,4,OpenDataSoft,data.rennesmetropole.fr,NaN,2023-04-14T00:00:00,0.56,0,0,0,0
3,64486c8ab82c9730d92ca03e,Voiries - Tours Métropole Val de Loire,voiries-tours-metropole-val-de-loire,NaN,http://www.data.gouv.fr/fr/datasets/voiries-to...,Tours Métropole Val de Loire,5cb6d4f78b4c4118e18f37d1,"Données géographique linéaire, une voie est un...",unknown,Licence Ouverte / Open Licence version 2.0,...,4,OpenDataSoft,data.tours-metropole.fr,NaN,2023-04-25T09:39:27,0.44,0,0,0,0
4,6447d43412d5f7f8f4970cb4,Plan de transport théorique,plan-de-transport-theorique,NaN,http://www.data.gouv.fr/fr/datasets/plan-de-tr...,TRENITALIA FRANCE,643ea16a8a200d21714dad13,Plan de transport théorique de Trenitalia France,daily,License Not Specified,...,0,NaN,NaN,NaN,NaN,0.33,0,0,0,0


Filtrage "intelligent" :)

In [163]:
# Filtrer les lignes selon vos critères et sélectionner les colonnes spécifiées
mask_org = data['organization'].str.startswith('Région') | data['organization'].str.startswith('Collectivité de Corse') # on pourrait le faire avec l'id de la region dans datagouv
mask_desc = data['description'].str.contains('conventions de subvention', case=False)
mask_title_1 = data['title'].str.contains('conventions de subvention', case=False)
mask_title_2 = data['title'].str.contains('subventions', case=False) & data['title'].str.contains('associations', case=False)
mask_title_3 = data['title'].str.contains('Subventions du Conseil Régional',case = False)
mask_title_4 = data['title'].str.contains('Interventions de la Région des Pays de la Loire',case = False)
mask_title_5 = data['title'].str.contains('SCDL - Subventions',case = False)
#mask_slug = data['slug'].str.contains('subventions', case=False)
#mask_tags = data['tags'].str.contains('subventions', case=False)

#data_filtered = data[mask_org & (mask_desc | mask_title | mask_slug | mask_tags)][['id', 'title', 'url', 'description','organization', 'frequency']]
data_filtered = data[mask_org & (mask_title_1 | mask_title_2 | mask_title_3 | mask_title_4 | mask_title_5 | mask_desc)][['id', 'title', 'url', 'description','organization','organization_id', 'frequency']]



In [164]:
# Obtenir toutes les instances uniques de la colonne "organization"
unique_orgs = data_filtered['organization'].unique()

# Imprimer la liste des instances uniques
print(unique_orgs)

data_filtered.to_csv('dataset_filtered_subventions.csv', index=False)
FileLink('dataset_filtered_subventions.csv')

['Région GRAND EST' 'Région des Pays de la Loire'
 'Région Centre-Val de Loire' 'Collectivité de Corse' 'Région Bretagne'
 'Région Occitanie / Pyrénées Méditerranée' 'Région Île-de-France']


/home/jupyter/dataset_filtered_subventions.csv

schema de donnees pour les subventions: https://git.opendatafrance.net/scdl/subventions

Commentaires sur chaque region et leur status
Ile-de-France  -- data OK schema pas OK

PACA -- data archives (Archives régionales Région Provence-Alpes-Côte-d'Azur vraiment galere)

Occitanie -- data OK schema OK

Nouvelle-Aquitaine -- R aucune donnee dans data.gouv c'est assez rare pour etre souligne 

Grand Est -- data OK schema OK (meme si manque des colonnes)

Bretagne -- OK schema OK

Normandie -- OK mais pas dans data.gouv API arcGIS https://services2.arcgis.com/OSTljxuSj3y4z219/arcgis/rest/services/Subventions_Région_Normandie_2021/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson a tester

Pays de la Loire -- Interventions de la Région des Pays de la Loire schema pas OK

Hauts-de-France -- pas OK (budget primitif et comptes administratifs -- mais pdf sans detail)

Centre-Val-de-Loire -- OK schema OK

Auvergne-Rhône-Alpes -- pas trouve

Corse -- OK

Bourgogne-Franche-Comté -- pas trouve

Reunion -- pas trouve (commande publique oui + budget)

La procedure ci-dessous fait la meme chose que celle sur les datasets de datagouv. Prend le lien de toutes les ressources de datagouv et les met dans une dataframe data_files

In [152]:
url = 'https://www.data.gouv.fr/fr/datasets/r/4babf5f2-6a9c-45b5-9144-ca5eae6a7a6d'

response = requests.get(url)

# vérifier le type de contenu de la réponse
content_type = response.headers.get('content-type')

if 'json' in content_type:
    # La réponse est au format JSON, on peut utiliser response.json()
    data_files = response.json()
    # code pour travailler avec les données JSON ici

elif 'csv' in content_type:
    # La réponse est au format CSV
    # Utiliser la fonction pandas.read_csv() pour lire les données CSV dans un dataframe
    data_files = pd.read_csv(url,delimiter=';')
    # code pour travailler avec le dataframe ici
    #print(data.head())
else:
    # Le type de contenu n'est pas géré
    print("Type de contenu non géré : ", content_type)

                         id  \
0  6448a8817c8f7ead28c148e0   
1  6448a87e3cbb852f0c4ff881   
2  644893ec16d85892112ca03c   
3  64486c8ab82c9730d92ca03e   
4  6447d43412d5f7f8f4970cb4   

                                               title  \
0  Lieux de médiation numérique sur le territoire...   
1  Lieux de médiation numérique sur le territoire...   
2          Parkings vélos gérés par Rennes Métropole   
3             Voiries - Tours Métropole Val de Loire   
4                        Plan de transport théorique   

                                                slug acronym  \
0  lieux-de-mediation-numerique-sur-le-territoire...     NaN   
1  lieux-de-mediation-numerique-sur-le-territoire...     NaN   
2        parkings-velos-geres-par-rennes-metropole-1     NaN   
3               voiries-tours-metropole-val-de-loire     NaN   
4                        plan-de-transport-theorique     NaN   

                                                 url  \
0  http://www.data.gouv.fr/fr/datas

In [165]:
data_files.head()

,dataset.id,dataset.title,dataset.slug,dataset.url,dataset.organization,dataset.organization_id,dataset.license,dataset.private,dataset.archived,id,...,format,mime,filesize,checksum.type,checksum.value,created_at,modified,downloads,harvest.created_at,harvest.modified_at
0,6448a8817c8f7ead28c148e0,Lieux de médiation numérique sur le territoire...,lieux-de-mediation-numerique-sur-le-territoire...,http://www.data.gouv.fr/fr/datasets/lieux-de-m...,Cartographie Nationale des lieux de médiation ...,636cd3d7336141e7d0577f56,Licence Ouverte / Open Licence version 2.0,False,False,6df37080-ea0f-458d-9270-9a7dc3e50abb,...,json,application/json,53253.0,sha1,14e352ec76566e9a80d77584523358206492bcb9,2023-04-26T06:28:51.439000,2023-04-26T06:28:52.434000,0,NaN,NaN
1,6448a8817c8f7ead28c148e0,Lieux de médiation numérique sur le territoire...,lieux-de-mediation-numerique-sur-le-territoire...,http://www.data.gouv.fr/fr/datasets/lieux-de-m...,Cartographie Nationale des lieux de médiation ...,636cd3d7336141e7d0577f56,Licence Ouverte / Open Licence version 2.0,False,False,5374b524-ab85-4457-9962-f3b279a240d7,...,json,application/json,80009.0,sha1,34069cab3414b1a42241db5ab8169c27523ae765,2023-04-26T06:28:51.359000,2023-04-26T06:28:52.490000,0,NaN,NaN
2,6448a8817c8f7ead28c148e0,Lieux de médiation numérique sur le territoire...,lieux-de-mediation-numerique-sur-le-territoire...,http://www.data.gouv.fr/fr/datasets/lieux-de-m...,Cartographie Nationale des lieux de médiation ...,636cd3d7336141e7d0577f56,Licence Ouverte / Open Licence version 2.0,False,False,87cbf19f-e738-4d9c-99ae-a6ee4de7caff,...,csv,text/csv,60259.0,sha1,97646a1557443789572e85597affff415c5772f6,2023-04-26T06:28:51.267000,2023-04-26T06:28:52.125000,0,NaN,NaN
3,6448a8817c8f7ead28c148e0,Lieux de médiation numérique sur le territoire...,lieux-de-mediation-numerique-sur-le-territoire...,http://www.data.gouv.fr/fr/datasets/lieux-de-m...,Cartographie Nationale des lieux de médiation ...,636cd3d7336141e7d0577f56,Licence Ouverte / Open Licence version 2.0,False,False,36992e97-7fc3-4a7e-9c1b-a932098212d2,...,json,application/json,78172.0,sha1,4995cdb31ceea1e0d61e0e24baa4b6192724cf37,2023-04-26T06:28:51.256000,2023-04-26T06:28:52.126000,0,NaN,NaN
4,6448a87e3cbb852f0c4ff881,Lieux de médiation numérique sur le territoire...,lieux-de-mediation-numerique-sur-le-territoire...,http://www.data.gouv.fr/fr/datasets/lieux-de-m...,Cartographie Nationale des lieux de médiation ...,636cd3d7336141e7d0577f56,Licence Ouverte / Open Licence version 2.0,False,False,143ed6c4-5306-4b10-b322-fd1bece63e46,...,json,application/json,63697.0,sha1,90972e85248e4f61ee65c96475dc11239ce7a6b1,2023-04-26T06:28:48.393000,2023-04-26T06:28:49.283000,0,NaN,NaN


In [166]:
#jointure entre table des datasets filtrees sur les bons themes et la table des fichiers
result = data_filtered.merge(data_files, left_on='id', right_on='dataset.id')

In [167]:
result.to_csv('files_filtered_subventions.csv', index=False)
FileLink('files_filtered_subventions.csv')

/home/jupyter/files_filtered_subventions.csv

In [168]:
# garder que les lignes ou il y a des formats csv (probablement a changer dans le futur)
files_filtered_subventions = result[result['format']=='csv']
files_filtered_subventions.head()

,id_x,title_x,url_x,description_x,organization,organization_id,frequency,dataset.id,dataset.title,dataset.slug,...,format,mime,filesize,checksum.type,checksum.value,created_at,modified,downloads,harvest.created_at,harvest.modified_at
1,6430af207f3e31d2c253867d,Données essentielles des conventions de subven...,http://www.data.gouv.fr/fr/datasets/1646230541...,Données essentielles des conventions de subven...,Région GRAND EST,55df259f88ee386e57a46ec2,NaN,6430af207f3e31d2c253867d,Données essentielles des conventions de subven...,1646230541247-1-7,...,csv,NaN,NaN,NaN,NaN,2023-04-07T00:00:00,2023-04-08T02:02:40.284000,0,2023-04-07T00:00:00,NaN
3,642cbb15b047f22ca7538678,Données essentielles des conventions de subven...,http://www.data.gouv.fr/fr/datasets/1646230541...,Données essentielles des conventions de subven...,Région GRAND EST,55df259f88ee386e57a46ec2,NaN,642cbb15b047f22ca7538678,Données essentielles des conventions de subven...,1646230541247-1-6,...,csv,NaN,NaN,NaN,NaN,2023-04-04T00:00:00,2023-04-05T02:04:37.379000,0,2023-04-04T00:00:00,NaN
4,63d85a7f4c2526cadb40440d,Interventions de la Région des Pays de la Loire,http://www.data.gouv.fr/fr/datasets/interventi...,"Liste des interventions (Aides, Appels à proje...",Région des Pays de la Loire,534fffa7a3a7292c64a780c5,unknown,63d85a7f4c2526cadb40440d,Interventions de la Région des Pays de la Loire,interventions-de-la-region-des-pays-de-la-loire,...,csv,text/csv,NaN,NaN,NaN,2023-01-31T01:02:07.319000,2023-04-21T21:11:05.875000,0,NaN,2023-04-21T21:11:05.875000
7,63d071ccf66c897ec1404414,Données essentielles des conventions de subven...,http://www.data.gouv.fr/fr/datasets/1646230541...,Données essentielles des conventions de subven...,Région GRAND EST,55df259f88ee386e57a46ec2,NaN,63d071ccf66c897ec1404414,Données essentielles des conventions de subven...,1646230541247-1-5,...,csv,NaN,NaN,NaN,NaN,2023-01-24T00:00:00,2023-01-25T01:03:24.644000,3,2023-01-24T00:00:00,NaN
9,63a24f0c207bdb78a99dde9e,Données essentielles des conventions de subven...,http://www.data.gouv.fr/fr/datasets/1646230541...,Données essentielles des conventions de subven...,Région GRAND EST,55df259f88ee386e57a46ec2,NaN,63a24f0c207bdb78a99dde9e,Données essentielles des conventions de subven...,1646230541247-1-4,...,csv,NaN,NaN,NaN,NaN,2022-12-20T00:00:00,2022-12-21T01:10:52.399000,2,2022-12-20T00:00:00,NaN


schema donne par https://git.opendatafrance.net/scdl/subventions

In [158]:
#nomAttribuant
#idAttribuant
#dateConvention
#referenceDecision
#nomBeneficiaire
#idBeneficiaire
#rnaBeneficiaire
#objet
#montant
#nature
#conditionsVersement
#datesPeriodeVersement
#idRAE
#notificationUE
#pourcentageSubvention
#dispositifAide

procedure pour regarder chaque ligne et telecharger les fichiers csv associes

In [157]:
def detect_delimiter(text, num_lines=5, delimiters=None):
    if delimiters is None:
        delimiters = [',', ';', '\t', '|']

    counts = {delimiter: 0 for delimiter in delimiters}
    
    for line_number, line in enumerate(StringIO(text)):
        if line_number >= num_lines:
            break
        for delimiter in delimiters:
            if delimiter in line:
                counts[delimiter] += 1
                    
    return max(counts, key=counts.get)

def load_csv_from_url(url):
    try:
        response = requests.get(url)
        content = response.text
        delimiter = detect_delimiter(content)
        df = pd.read_csv(StringIO(content), delimiter=delimiter, encoding='utf-8')
        return df
    except:
        print(f"Erreur lors du téléchargement du fichier CSV à l'URL : {url}")
        return None

# Remplacez ces liens par les liens réels vers les fichiers CSV
links_df = files_filtered_subventions

#je me suis apercu que tout est etait lowercase dans beaucoup de regions
desired_columns = [
    'nomattribuant', 'idattribuant', 'dateconvention', 'referencedecision',
    'nombeneficiaire', 'idbeneficiaire', 'rnabeneficiaire', 'objet', 'montant',
    'nature', 'conditionsversement', 'datesperiodeversement', 'idrae',
    'notificationue', 'pourcentagesubvention', 'dispositifaide'
]

main_df = pd.DataFrame(columns=desired_columns)

for index, row in links_df.iterrows():
    csv_link = row['url_y']
    organization_id = row['organization_id']
    organization = row['organization']
    
    csv_df = load_csv_from_url(csv_link)
    
    if csv_df is not None:
        # Ajouter les colonnes 'organization_id' et 'organization' à csv_df
        csv_df['organization_id'] = organization_id
        csv_df['organization'] = organization
        
        csv_df = csv_df[[col for col in csv_df.columns if col in desired_columns + ['organization_id', 'organization']]]
        main_df = main_df.append(csv_df, ignore_index=True)

#print(main_df)


Erreur lors du téléchargement du fichier CSV à l'URL : https://grandestprod.data4citizen.com/data4citizen/sites/default/files/dataset/2022/11/23/ecf06d2f-12b9-4972-b50d-dca572fd509e/subvention_region_grandest.csv
Erreur lors du téléchargement du fichier CSV à l'URL : https://grandestprod.data4citizen.com/data4citizen/sites/default/files/dataset/2022/11/18/4846b2ca-0fee-4297-a4ae-96280c453908/subvention_region_grandest.csv
Erreur lors du téléchargement du fichier CSV à l'URL : https://grandestprod.data4citizen.com/data4citizen/sites/default/files/dataset/2022/03/15/86423489-e587-4d46-8d11-460679dbbbfe/subvention_region_grandest.csv
            nomattribuant    idattribuant       dateconvention  \
0       RÃ©gion Grand Est  20005226400013  2019-01-21 00:00:00   
1       RÃ©gion Grand Est  20005226400013  2019-11-22 00:00:00   
2       RÃ©gion Grand Est  20005226400013  2019-11-22 00:00:00   
3       RÃ©gion Grand Est  20005226400013  2019-11-22 00:00:00   
4       RÃ©gion Grand Est  2000

In [169]:
# Supprimer les lignes en double de main_df
unique_main_df = main_df.drop_duplicates()

unique_main_df.shape

(98439, 18)

In [170]:
unique_main_df.head()
unique_main_df.to_csv('detailed_subventions_all_regions.csv', index=False)
FileLink('detailed_subventions_all_regions.csv')

/home/jupyter/detailed_subventions_all_regions.csv